In [ ]:
!pip install -q transformers datasets evaluate
!pip install -q python-dotenv

In [ ]:
# Подключение данных гугл диска
from google.colab import drive
drive.mount('/content/drive')

# from dotenv import load_dotenv  ## it was needed for WANDB login, but not anymore
# load_dotenv('/content/drive/MyDrive/env.sh')

Mounted at /content/drive


<a id='trainer'></a>

Данные о позитивных/негативных отзывах, лежат тут https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/data

Если прогоняете на Google colab, после скачивания надо загрузить их сюда

In [161]:
from IPython.display import display
from tqdm import tqdm
import typing as t
import numpy as np
import pandas as pd
import torch
torch.cuda.empty_cache()
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers.models.bert.modeling_bert import BertForSequenceClassification
import evaluate


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def transform_label(label):
    return 1 if label == 'positive' else 0


def compute_accuracy(dataloader: DataLoader, model: BertForSequenceClassification, label_key: str='label') -> t.Dict[str, float]:

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

  metric = evaluate.load("accuracy")
  model.to(device)
  model.eval()
  for batch in tqdm(dataloader):
      labels = batch[label_key]
      x_batch = {k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'token_type_ids', 'attention_mask']}
      with torch.no_grad():
          outputs = model(**x_batch)

      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)
      metric.add_batch(predictions=predictions, references=labels)

  return metric.compute()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

def transform_label(label):
    return 1 if label == 'positive' else 0

# imdb_dataset = pd.read_csv('/content/drive/MyDrive/Data/IMDB/IMDB Dataset.csv')
imdb_dataset = pd.read_csv('https://drive.google.com/uc?id=1Drz1blRg0AsPWMB7oGQNsbyTpNMObDrC')
imdb_dataset = imdb_dataset.rename(columns={'review': 'text', 'sentiment': 'label'})
imdb_dataset['label'] = imdb_dataset['label'].apply(transform_label)
train, test = train_test_split(imdb_dataset, test_size=0.2)

In [ ]:
print("="*50 + 'Positive' + '='*50)
for text in imdb_dataset.loc[(imdb_dataset['label'] == 1)]['text'].sample(n=5).values:
  display(text)
  print()


print("="*50 + 'Negative' + '='*50)
for text in imdb_dataset.loc[(imdb_dataset['label'] == 0)]['text'].sample(n=5).values:
  display(text)
  print()



==================================================Positive==================================================


'In one of the many Bugs Bunny-Daffy Duck cartoons, Elmer Fudd is out hunting, and Daffy tries to get him to shoot Bugs. Needless to say, Bugs has his own agenda. Moreover, "Rabbit Seasoning" makes interesting use of word order and pronouns (warning: it just might hilariously and royally mess up your speech).<br /><br />I think that probably my favorite aspect of this cartoon is the costumes worn by Bugs and Daffy. One of them seems like it would have been risqué for 1952 (especially in a cartoon), but they pull it off perfectly, as they always did. All in all, this just goes to show what geniuses the people behind these cartoons were.'

'I would like to say that I absolutely loved Campfire Tales! To me, it was one of the original horror flicks with a twisted ending. As if the contents of the movie weren\'t scary or weird enough, you have the ending. It\'s a very awesome movie and I\'m so happy that it\'s being released on DVD on August 30th.I will not hesitate to get my copy on that day.I don\'t think the movie received all the credit or recognition it should have, because all these other movies came out shortly after and was acknowledge, but if it weren\'t for my brother telling me about the movie, I wouldn\'t have known. Which stinks because in my opinion it\'s far better than any "Scream", "I know what you did Last Summer", or any other horror for that time. So I hope that you are able to see the movie yourself and at least be able to see the difference. I loved it, and being a horror movie junkie, Campfire Tales was like dessert for me!'

'For many the hit series was ten years of pitch black humour loaded with affectionate parodies of classic films and a hilarious assortment of over a hundred characters with instantly recognisable catchphrases. Few shows have survived transition from radio to TV to stage show to film but The League of Gentlemen have achieved it with suitable aplomb.<br /><br />The talented writer/performers had initially envisioned a Monty Python style medieval adventure, but as soon as writing began they soon realised that the characters they have lived with had become very real and deserved better. With that, the Royston Vasey folk realise their very existence is under threat as the writers decide to disregard the fictitious town and work on a 17th Century romp instead.<br /><br />With the exception of Michael Sheen playing much unseen League member Jeremy Dyson, The League play pretty unlikeable caricatures of their real life personae as well as the familiar faces of Tubbs ("I made a little brown fis

"Dirty Dancing one of my MOST favorite movies. I've only watched it two times on ABC because I haven't had the chance to buy it or rent it from Blockbuster. I had no idea Jennifer Grey was 27 at the time she made the movie, because she was very convincing as a teenager. Compared to some women in Hollywood she has a very flat chest, which is why I was fooled so easily about her age. So both physically and emotionally, Grey pulls off playing a teen very well. I also loved the dancing--who WOULDN'T? Both times I've watched Dirty Dancing, I keep wanting to look up dance classes. I also love the soundtrack, and I do recognize some of the songs from when I was eight or nine. I would LOVE to be able to watch this in my drama class, and I'm going to ask my teacher at some point if there's any part of the movie he can use for educational purposes. It's much better than the stuff he made us watch last year. And the ending was absolutely FANTASTIC. That's one of the best moments in the film.<br /

...

"Slashers.....well if you like horrors its definitely one to see, otherwise don't even bother.It is completely obvious that this film has an extremely low budget, For instance it looks as if the entire film has been shot in a warehouse somewhere, and on numerous occasions you will see the mike boom shadow and the camera mans shadow, trust me you wont need to look for them.Also try to ignore the cheesy actors, if thats what you call them!!The basic outline is a few people decide to go on a game show where they have to survive a night in a big maze due to their being 3 killers on the loose and whoever live's at the end gets rich. Now there is something about this film that keeps you watching and rarely do you find that with a cheap budget horror these days,For example when i watched it i thought to my self i would'nt mind having a go at this game! especially for $12.000.000. so anyway i would recommend you watch it and make up your own mind."

'This movie was different from most of Jimmy Cangey\'s films of the 1930s in that it was NOT done by Warner Brothers/First National, but was a loan-out to a smaller studio. Because it was a "poverty row" studio, the production values are lower than you might be used to seeing with Cagney films. Plus, the plot is certainly one of the strangest I have seen. Instead of him being a gangster, he was a good guy in this one--fighting for the law. This isn\'t all that unusual because Cagney frequently played lawmen--such as an OSS leader (the forefather of the CIA) of FBI agent. BUT, to make him an investigator for the Bureau of Weights and Measures was indeed odd--especially since, at times, he acted pretty much the same way he did in the movie G-MEN! <br /><br />All in all, a time passer and that\'s about it.<br /><br />Finally, the videotape I saw this on from Memory Lane Video was perhaps one of the poorest I have ever encountered. The sound was terrible and scratchy and the print looked v

"Being a fan of silent films, I looked forward to seeing this picture for the first time. I was pretty disappointed. <br /><br />As has been mentioned, the film seems to be one long, long, commercial for the Maxwell automobile. <br /><br />Perhaps if the chase scene was about half the length that it is, I may have enjoyed the film more. But it got old very fast. And while I recognize that reality is stretched many times in films, without lessening a viewer's enjoyment, what was with the Mexican bandits? I mean, they are chasing a car through the mountains, a car that most of the time is moving at about one mile per hour, yet they can't catch up to it?"

"After a day at work, I sat down to relax and turned on the movie channels. The movie came up on the guide and sounded interesting so I tuned in just before it started. The first 30 minutes were enough to make me interested, but the lack of acting ability in Jamie Foxx and the slow plot movement made me want to get up and find food during the movie. If there is any credit to be given for acting in this movie it should go to David Morse who at least tries to make the movie interesting. All in all, don't plan on impressing your friends by picking this one as a renter for a movie night."

'I am not a big fan of the Spielberg/Cruise version of this film. And so I must throw in with the more humble Latt/Howel version. C Thomas Howel had more heart and more sympathy that Cruise in the lead role (at least in my opinion). Now this is hard to imagine until you strip away everything thing in the Spielberg version that cost more than a thousand dollars. There would be nothing left, no special effects, no sets, no Cruise. Because I doubt that anything cost more than a cool grand in the David Michael Latt version. So, comparing apples to turnips I guess I have to go with the turnips. At least it gives independent movies a shot at the epic science fiction disaster market.'

In [ ]:
dataset = load_dataset('csv', data_files={'train': "/content/imdb_train.csv", 'test': "/content/imdb_test.csv"})
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [176]:
# tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels") # because outputs.loss expects 'labels' column
tokenized_datasets.set_format("torch")

In [177]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10_000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10_000))

In [178]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=16)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=16)

In [164]:
compute_accuracy(dataloader=eval_dataloader, model=model, label_key='labels')

100%|██████████| 157/157 [04:56<00:00,  1.89s/it]


{'accuracy': 0.4953}

In [184]:
import torch
from torch.optim import AdamW

from transformers import get_scheduler


optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
torch.cuda.empty_cache()
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

<Tip>

Get free access to a cloud GPU if you don't have one with a hosted notebook like [Colaboratory](https://colab.research.google.com/) or [SageMaker StudioLab](https://studiolab.sagemaker.aws/).

</Tip>

Great, now you are ready to train! 🥳

### Training loop

To keep track of your training progress, use the [tqdm](https://tqdm.github.io/) library to add a progress bar over the number of training steps:

In [185]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in tqdm(range(num_epochs)):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'token_type_ids', 'attention_mask', 'labels']}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1875 [00:00<?, ?it/s]

In [187]:
# Directory to save the model
save_path = "/content/drive/MyDrive/Data/IMDB/bert_finetuned.pth"
model.to('cpu')
# Save the trained model
model.save_pretrained(save_path)

In [241]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained(
    '/content/drive/MyDrive/Data/IMDB/bert_finetuned.pth'
)
finetuned_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [242]:
compute_accuracy(dataloader=eval_dataloader, model=finetuned_model, label_key='labels')

  0%|          | 0/625 [00:00<?, ?it/s]

{'accuracy': 0.9229}

In [237]:
text, label = test[['text', 'label']].sample().values[0]
print("text:")
display(text)
print(f"\nLABEL: {'Positive' if label == 1 else 'Negative'} (label={label})")

text:


"I went to see this film over Matchstick Men, in fact buying the tickets to Matchstick Men and going to the other, because it looked like a fun movie with action, romance, thrills, jungles, and exotic locations. They had all that but so do a lot of movies with a conception of story.<br /><br />All I can say is WHY WHY WHY WHY did they not just make it a straight narrative instead of some sappy flashback story.<br /><br />Here is all the movies from what I've seen the film was derived from: Of course, Indiana Jones and Romancing the Stone, but also True Lies, Proof of Life, that old 80s Tom Selleck movie, Bananas (Woody Allen), and Hero (from the use of digital extras).<br /><br />PS the only scene in the movie that was cool is when the central character finds her room blown up."


LABEL: Negative (label=0)


In [238]:
logits = model(**tokenizer(text, padding="max_length", truncation=True, return_tensors='pt')).logits
print(f'Prediction: {"Positive" if logits.argmax(axis=1)[0] == 1 else "Negative"}')
display(logits)

Prediction: Negative


tensor([[ 2.4752, -1.2335]], grad_fn=<AddmmBackward0>)

<a id='additional-resources'></a>